# Notebook

## Init

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

## Cuda

In [ ]:
torch.__version__, torch.cuda.is_available(), torch.version.cuda

In [ ]:
cuda_id = torch.cuda.current_device()

In [ ]:
torch.cuda.current_device(), torch.cuda.get_device_name(cuda_id)

## Dataset

In [ ]:
batch_size_train = 64
batch_size_test = 1000
NUM_WORKERS=1
ngpu = 1 
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
train_data = torchvision.datasets.MNIST('./data/', train=True, download=True,
                             transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST('./data/', train=False, download=True,
                             transform=torchvision.transforms.ToTensor())

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size_train, shuffle=True, num_workers=NUM_WORKERS)

test_loader = DataLoader(test_data, batch_size=batch_size_test, shuffle=True, num_workers=NUM_WORKERS)

In [ ]:
plt.imshow(train_data.data[0]) #use cmap='gray' to view in grayscale
plt.title('%i' % train_data.targets[0])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
ex = enumerate(train_loader)
batch_idx, (ex_data, ex_targets) = next(ex)

In [ ]:
ex_data.shape

In [ ]:
plt.imshow(ex_data[0][0], cmap='gray', interpolation='none')
plt.title('%i' % ex_targets[0])
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
import torch.nn as nn
## Now let's work on the neural network
## Curren NN drawing in data file